In [ ]:
def pertubate(self):
    # Başlangıç ayarları
    accus, mafs = list(), list()  # Her epsilon için doğruluk ve macro F1 skorlarını saklayacak listeler
    epsilon_values = np.linspace(0, 0.1, 15)  # Epsilon değerleri (0'dan 0.1'e kadar 15 adımda)

In [ ]:
    # Epsilon değerleri üzerinde döngü başlat
    for epsilon in tqdm.tqdm(epsilon_values):
        prediction_ = []
        real_ = []

In [ ]:
        # Test veri kümesi üzerinde döngü
        for data, label in self.test:
            # Giriş verilerini GPU'ya aktar (mevcutsa) ve label verisini sakla
            data = data.cuda() if tr.cuda.is_available() else data
            label = label.cuda() if tr.cuda.is_available() else label
            real_.append(label)

In [ ]:
            if epsilon != 0:
                data.requires_grad = True  # Gradyan hesaplamak için giriş verisini aktif hale getir
                
                # Tahmin yap ve kaybı hesapla
                prediction = self.net(data)
                loss = self.loss_function(prediction, label)
                loss.backward()  # Geriye doğru yayılım ile gradyanı hesapla

                # Sonsuz norm için pertürbasyon hesapla
                perturbation = epsilon * data.grad.data.sign()
                
                # Veriyi pertürbasyon ile boz
                x_adv = tr.clamp(data + perturbation, -1, 1)
            else:
                x_adv = data  # Epsilon sıfırsa, orijinal veriyi kullan

In [ ]:
            # Pertürbasyon uygulanmış veri ile tahmin yap
            prediction = self.net(x_adv)
            prediction_.append(prediction.detach().cpu())

In [ ]:
        # Tahmin ve gerçek değerleri birleştir
        prediction_ = tr.cat(prediction_, 0)
        real_ = tr.cat(real_, 0)

        # Doğruluk ve macro F1 skoru hesapla
        prediction_ = tr.argmax(prediction_, -1)
        accu = self.accu_(prediction_, real_)
        maf1 = self.maf1(predicted=prediction_, real=real_)
        
        # Hesaplanan skorları sakla
        accus.append(accu)
        mafs.append(maf1)

In [ ]:
import numpy as np
import tqdm
import torch as tr

def pertubate(self):
    # Başlangıç ayarları
    accus, mafs = list(), list()  # Her epsilon için doğruluk ve macro F1 skorlarını saklayacak listeler
    epsilon_values = np.linspace(0, 0.1, 15)  # Epsilon değerleri (0'dan 0.1'e kadar 15 adımda)

    # Epsilon değerleri üzerinde döngü başlat
    for epsilon in tqdm.tqdm(epsilon_values):
        prediction_ = []
        real_ = []

        # Test veri kümesi üzerinde döngü
        for data, label in self.test:
            # Giriş verilerini GPU'ya aktar (mevcutsa) ve label verisini sakla
            data = data.cuda() if tr.cuda.is_available() else data
            label = label.cuda() if tr.cuda.is_available() else label
            real_.append(label)

            if epsilon != 0:
                data.requires_grad = True  # Gradyan hesaplamak için giriş verisini aktif hale getir
                
                # Tahmin yap ve kaybı hesapla
                prediction = self.net(data)
                loss = self.loss_function(prediction, label)
                loss.backward()  # Geriye doğru yayılım ile gradyanı hesapla

                # Sonsuz norm için pertürbasyon hesapla
                perturbation = epsilon * data.grad.data.sign()
                
                # Veriyi pertürbasyon ile boz
                x_adv = tr.clamp(data + perturbation, -1, 1)
            else:
                x_adv = data  # Epsilon sıfırsa, orijinal veriyi kullan

            # Pertürbasyon uygulanmış veri ile tahmin yap
            prediction = self.net(x_adv)
            prediction_.append(prediction.detach().cpu())

        # Tahmin ve gerçek değerleri birleştir
        prediction_ = tr.cat(prediction_, 0)
        real_ = tr.cat(real_, 0)

        # Doğruluk ve macro F1 skoru hesapla
        prediction_ = tr.argmax(prediction_, -1)
        accu = self.accu_(prediction_, real_)
        maf1 = self.maf1(predicted=prediction_, real=real_)
        
        # Hesaplanan skorları sakla
        accus.append(accu)
        mafs.append(maf1)

    # Sonuçları göster
    print("Doğruluk Skorları:", accus)
    print("Macro F1 Skorları:", mafs)